In [1]:
import torch
import numpy as np
import plyfile
import os
import laspy

In [2]:
num_classes = 5

# 0- nth, 1 - pole, 2 - road line, 3 - road, 4 - traffic sign 
class Paris():
    def __init__(self, path) -> None:
        if not os.path.exists(path):
            return
        
        ply = plyfile.PlyData.read(path)
        print(ply)
        
        x = torch.from_numpy(np.asarray(ply.elements[0]['X']).copy())
        y = torch.from_numpy(np.asarray(ply.elements[0]['Y']).copy())
        z = torch.from_numpy(np.asarray(ply.elements[0]['Z']).copy())
        intensity = torch.from_numpy(np.asarray(ply.elements[0]['Intensity']).copy()) / 255.0
        self.points = torch.stack([x, y, z, intensity], dim=1)
    
        self.frame_index = torch.from_numpy(np.asarray(ply.elements[0]['Frame_Index'], dtype=np.int32).copy()) - 1
        self.num_frames = self.frame_index[-1]
        
        self.gps_time = torch.from_numpy(np.asarray(ply.elements[0]['GPS_Time'], dtype=np.int32).copy())
        
        
    def get_points(self, idx) -> torch.Tensor:
        return self.points[self.frame_index == idx]

In [3]:
paris = Paris('data/Lille1/Lille_0.ply')

ply
format binary_little_endian 1.0
element vertex 10000000
property double X
property double Y
property double Z
property uchar Intensity
property float Scan_Angle
property double GPS_Time
property double X_Sensor_Position
property double Y_Sensor_Position
property double Z_Sensor_Position
property float Vertical_Laser_Angle
property uchar Laser_Index
property uint Frame_Index
end_header


In [4]:
paris.frame_index

tensor([  0,   0,   0,  ..., 215, 215, 215], dtype=torch.int32)

In [5]:
from vision.PointTransformer import PointTransformerSeg

seg_model = PointTransformerSeg(4, num_classes)
seg_model.load_state_dict(torch.load('data/point_transformer_seg.pt'))
seg_model.cuda()
seg_model.eval()

PointTransformerSeg(
  (mlp1): SingleMLP(
    (fc): Linear(in_features=4, out_features=32, bias=True)
    (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (transformer1): PointTransformerBlock(
    (transformerLayer): PointTransformerLayer(
      (query): Linear(in_features=32, out_features=32, bias=True)
      (key): Linear(in_features=32, out_features=32, bias=True)
      (value): Linear(in_features=32, out_features=32, bias=True)
      (fc_delta1): Linear(in_features=3, out_features=3, bias=True)
      (fc_delta2): Linear(in_features=3, out_features=32, bias=True)
      (bn_delta): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc_gamma1): Linear(in_features=32, out_features=32, bias=True)
      (fc_gamma2): Linear(in_features=32, out_features=32, bias=True)
      (bn_gamma1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn_gamma2): BatchNorm2d(32, eps=1e-05, m

In [6]:
def normalize_points(points):
    points[:,:,:3] -= torch.mean(points[:,:,:3], dim=1)
    points[:,:,:3] /= torch.max(torch.abs(points[:,:,:3]), dim=1).values
    return points

def create_las(path:str, paris:Paris, model):
    # with torch.no_grad():
    points_list = []
    classes_list = []
    
    for i in range(paris.num_frames):
        points = paris.get_points(i)
        points_list.append(points)
        points = points.unsqueeze(0).cuda().type(torch.float32)
        points = normalize_points(points)
        
        with torch.no_grad():
            prediction = model(points)
            prediction = prediction.transpose(1, 2)
            prediction = torch.nn.functional.softmax(prediction, dim=1)
            prediction = prediction.argmax(dim=1).squeeze().detach().cpu()
            print(prediction.shape)
            print(prediction[prediction > 0])
            classes_list.append(prediction)
            
    points = torch.concatenate(points_list, dim=0).numpy()
    classes = torch.concatenate(classes_list, dim=0).numpy()
    intensity = points[:, 3] * 255
    intensity = intensity.astype(np.uint8)
    
    header = laspy.LasHeader(point_format=3, version="1.4")
    header.offsets = np.min(points[:,:3], axis=0)
    header.scales = np.array([0.01, 0.01, 0.01])
    # header.vlrs
    
    las = laspy.LasData(header)
    las.x = points[:,0]
    las.y = points[:,1]
    las.z = points[:,2]
    las.intensity = intensity
    las.classification = classes
    # las.gps_time = paris.gps_time
    # las.point_source_id = paris.frame_index
    
    las.write(path)
    print(f'{points.shape[0]} points were saved on {path}.')

In [7]:
create_las('./test.las', paris, seg_model)

torch.Size([49241])
tensor([4, 4, 4,  ..., 4, 4, 4])
torch.Size([49477])
tensor([4, 4, 4,  ..., 1, 1, 1])
torch.Size([49729])
tensor([4, 4, 4,  ..., 1, 1, 1])
torch.Size([49898])
tensor([1, 1, 1,  ..., 1, 1, 1])
torch.Size([49922])
tensor([1, 1, 1,  ..., 1, 1, 1])
torch.Size([49914])
tensor([1, 1, 1,  ..., 4, 1, 4])
torch.Size([49941])
tensor([1, 1, 1,  ..., 1, 1, 1])
torch.Size([49003])
tensor([1, 1, 1,  ..., 1, 1, 1])
torch.Size([49147])
tensor([1, 1, 1,  ..., 1, 1, 1])
torch.Size([49539])
tensor([1, 1, 1,  ..., 1, 1, 1])
torch.Size([49746])
tensor([1, 1, 3,  ..., 1, 1, 1])
torch.Size([49520])
tensor([3, 3, 3,  ..., 1, 1, 1])
torch.Size([49132])
tensor([3, 3, 3,  ..., 4, 4, 4])
torch.Size([48571])
tensor([2, 3, 2,  ..., 4, 4, 4])
torch.Size([48608])
tensor([3, 3, 3,  ..., 4, 4, 4])
torch.Size([48916])
tensor([3, 3, 3,  ..., 4, 4, 4])
torch.Size([49269])
tensor([3, 3, 3,  ..., 4, 4, 4])
torch.Size([49480])
tensor([1, 3, 3,  ..., 4, 4, 4])
torch.Size([49148])
tensor([3, 3, 3,  ..., 4, 